# Задание

Получите данные по энергопотреблению первых 20 зданий (building_id от 0 до 19).

Заполните отсутствующие значения по погоде интерполяционными данными.

Разделите данные на обучающие/проверочные в пропорции 80/20.

Постройте и найдите общее качество модели линейной регрессии, построенной по часам для каждого из первых 20 зданий по следующим параметрам: air_temperature, dew_temperature, cloud_coverage, wind_speed, precip_depth_1_hr, sea_level_pressure, is_holiday. Всего требуется построить 480 моделей линейной регрессии, вычислить по ним проверочные значения энергопотребления и получить итоговую оценку качества такой модели.

Для расчета последнего параметра (is_holiday) используйте график публичных выходных в США: USFederalHolidayCalendar

Исходные данные:
* http://video.ittensive.com/machine-learning/ashrae/building_metadata.csv.gz
* http://video.ittensive.com/machine-learning/ashrae/weather_train.csv.gz
* http://video.ittensive.com/machine-learning/ashrae/train.0.csv.gz

In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

import matplotlib.pyplot as plt
from matplotlib.pyplot import rcParams
rcParams['figure.figsize'] = 12, 6

buildings = pd.read_csv("http://video.ittensive.com/machine-learning/ashrae/building_metadata.csv.gz")
weather = pd.read_csv("http://video.ittensive.com/machine-learning/ashrae/weather_train.csv.gz")
energy = pd.read_csv("http://video.ittensive.com/machine-learning/ashrae/train.0.csv.gz")

### Получите данные по энергопотреблению первых 20 зданий (building_id от 0 до 19).

In [2]:
energy = energy[energy['building_id'] < 20]

### Заполните отсутствующие значения по погоде интерполяционными данными.

In [3]:
def reduce_mem_usage(df: pd.DataFrame):
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if str(col_type)[:5] == 'float':
            c_min = df[col].min()
            c_max = df[col].max()
            if c_min > np.finfo('f2').min and c_max < np.finfo('f2').max:
                df[col] = df[col].astype(np.float16)
            elif c_min > np.finfo('f4').min and c_max < np.finfo('f4').max:
                df[col] = df[col].astype(np.float32)
            else:
                df[col] = df[col].astype(np.float64)
        elif str(col_type)[:3] == 'int':
            c_min = df[col].min()
            c_max = df[col].max()
            if c_min > np.iinfo("i1").min and c_max < np.iinfo("i1").max:
                df[col] = df[col].astype(np.int8)
            elif c_min > np.iinfo("i2").min and c_max < np.iinfo("i2").max:
                df[col] = df[col].astype(np.int16)
            elif c_min > np.iinfo("i4").min and c_max < np.iinfo("i4").max:
                df[col] = df[col].astype(np.int32)
            elif c_min > np.iinfo("i8").min and c_max < np.iinfo("i8").max:
                df[col] = df[col].astype(np.int64)
        elif col == 'timestamp':
            df[col] = pd.to_datetime(df[col])
        elif str(col_type)[:8] != 'datetime':
            df[col] = df[col].astype('category')
    
    end_mem = df.memory_usage().sum() / 1024**2
    print(
        'Потребление памяти меньше на ',
        round(start_mem - end_mem, 2),
        ' Мб (-',
        round(100 * (start_mem - end_mem) / start_mem, 1),
        '%)',
        sep=''
    )
    return df


def show_inf_and_na(df: pd.DataFrame):
    pd.set_option('use_inf_as_na', True)
    for col in df.columns:
        print(col, 'Inf+NaN:', df[col].isnull().sum())


energy = pd.merge(left=energy, right=buildings, how='left', left_on='building_id', right_on='building_id')
energy = pd.merge(
    left=energy.set_index(['timestamp', 'site_id']), 
    right=weather.set_index(['timestamp', 'site_id']),
    how='left', left_index=True, right_index=True
)
energy.reset_index(inplace=True)
del weather
del buildings
energy = energy[energy['meter_reading'] > 0]
energy = reduce_mem_usage(energy)
show_inf_and_na(energy)

Потребление памяти меньше на 8.9 Мб (-63.2%)
timestamp Inf+NaN: 0
site_id Inf+NaN: 0
building_id Inf+NaN: 0
meter Inf+NaN: 0
meter_reading Inf+NaN: 0
primary_use Inf+NaN: 0
square_feet Inf+NaN: 0
year_built Inf+NaN: 0
floor_count Inf+NaN: 108573
air_temperature Inf+NaN: 0
cloud_coverage Inf+NaN: 45368
dew_temperature Inf+NaN: 0
precip_depth_1_hr Inf+NaN: 0
sea_level_pressure Inf+NaN: 569
wind_direction Inf+NaN: 3515
wind_speed Inf+NaN: 0


In [4]:
energy["precip_depth_1_hr"] = energy["precip_depth_1_hr"].map(lambda x: x if x > 0 else 0)
columns_for_interpolation = [
    'air_temperature', 'cloud_coverage', 'dew_temperature', 'precip_depth_1_hr', 'sea_level_pressure', 'wind_direction'
]
for column in columns_for_interpolation:
    energy[column] = energy[column].interpolate(limit_direction='both', kind='cubic')

show_inf_and_na(energy[columns_for_interpolation])

air_temperature Inf+NaN: 0
cloud_coverage Inf+NaN: 0
dew_temperature Inf+NaN: 0
precip_depth_1_hr Inf+NaN: 0
sea_level_pressure Inf+NaN: 0
wind_direction Inf+NaN: 0


### Сформировать признак is_holiday.

In [5]:
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar

us_holidays = calendar().holidays(start=energy['timestamp'].dt.date.min(), end=energy['timestamp'].dt.date.max())
energy['is_holiday'] = energy['timestamp'].isin(us_holidays).astype('int8')
energy['is_holiday'].sum()

140

### Разделите данные на обучающие/проверочные в пропорции 80/20.

In [6]:
energy_train, energy_test = train_test_split(energy, test_size=.2)
energy_train.shape, energy_test.shape

((86858, 17), (21715, 17))

### Постройте и найдите общее качество модели линейной регрессии, построенной по часам для каждого из первых 20 зданий.

In [7]:
features = [
    'air_temperature', 'dew_temperature', 'cloud_coverage', 'wind_speed',
    'precip_depth_1_hr', 'sea_level_pressure', 'is_holiday'
]


def generate_hourly_models(df: pd.DataFrame, x_features, y_feature='meter_reading'):
    hourly_models = dict()
    for hour in range(0, 24):
        hour_data = df[df['timestamp'].dt.hour == hour]
        hourly_models[hour] = LinearRegression().fit(hour_data[x_features].values, hour_data[y_feature].values)
    return hourly_models


building_models = dict()
for building_id in range(0, 20):    
    building_models[building_id] = generate_hourly_models(energy_train[energy_train['building_id'] == building_id], features)

amount = 0
for i in range(len(building_models)):
    amount += len(building_models[i])
print('Всего моделей:', amount)

Всего моделей: 480


### Получить итоговую оценку качества.

In [8]:
def calculate_squared_error(row):
    x = np.array(row[features]).reshape(1, -1)
    prediction = building_models[row.building_id][row.timestamp.hour].predict(x)[0]
    if prediction < 0:
        prediction = 0
    row['prediction'] = prediction
    row['squared_error'] = (np.log(row.meter_reading + 1) - np.log(prediction + 1))**2
    return row


energy_test = energy_test.apply(calculate_squared_error, axis='columns')
rmsle = np.sqrt(energy_test['squared_error'].sum() / len(energy_test))
print('Качество линейной регрессии:', rmsle)

Качество линейной регрессии: 0.27987732701602697


### Чему равна метрика качества полученной модели с точностью до десятых?

In [9]:
print('Качество линейной регрессии:', round(rmsle, 1))

Качество линейной регрессии: 0.3
